## So prometheus broke after usign GENIE one time 
And the error it threw was 

    return int_type_to_str[self.int_type]
KeyError: 1000000012

meaning 1000000012 particle wasnt defined in the translator list from particle type to ppc loss type. And the reason I didn't think to define a loss type for 1000000012 is because it is a .... sterile neutrino ???

what's that doing here, let us investigate:


In [91]:
import numpy as np
import matplotlib.pyplot as plt

In [92]:
import os
import ctypes
import sys


modules_to_clean = []
for name in list(sys.modules.keys()):
    if any(substring in name for substring in ['h5py', 'prometheus', '_hdf5', '_h5py', '_errors']):
        modules_to_clean.append(name)

for mod in modules_to_clean:
    if mod in sys.modules:
        del sys.modules[mod]

intel_paths = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64",
]

current_ld_path = os.environ.get('LD_LIBRARY_PATH', '')
new_ld_path = ':'.join(intel_paths + [current_ld_path])
os.environ['LD_LIBRARY_PATH'] = new_ld_path


intel_libs = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so",
]

for lib_path in intel_libs:
    try:
        ctypes.CDLL(lib_path, mode=ctypes.RTLD_GLOBAL)
        print(f"Successfully preloaded: {lib_path}")
    except Exception as e:
        print(f"Failed to preload {lib_path}: {e}")

sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus')
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples')

try:
    import h5py
    print("✓ h5py imported successfully")
except Exception as e:
    print(f"✗ h5py import failed: {e}")

try:
    from prometheus import Prometheus, config
    print("✓ prometheus imported successfully!")
except Exception as e:
    print(f"✗ prometheus import failed: {e}")
    import traceback
    traceback.print_exc()

2025-05-14 02:06:03,952 - h5py._conv - DEBUG - Creating converter from 7 to 5
2025-05-14 02:06:03,952 - h5py._conv - DEBUG - Creating converter from 5 to 7
2025-05-14 02:06:03,953 - h5py._conv - DEBUG - Creating converter from 7 to 5
2025-05-14 02:06:03,953 - h5py._conv - DEBUG - Creating converter from 5 to 7


Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so
✓ h5py imported successfully
✓ prometheus imported successfully!


In [53]:

from prometheus import Prometheus, config

In [54]:
import pandas as pd
import numpy as np
import time
import sys
import argparse
import os
import logging
from datetime import datetime

sys.path.append('../')
from prometheus import Prometheus, config
import prometheus
import gc

# Import our utility functions
from genie_parser_injection import parse_and_convert_genie

from inject_in_cylinder import inject_particles_in_cylinder
from rotate_particles import rotate_particles_final

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
RESOURCE_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/resources/"
OUTPUT_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/genie_examples/output"


In [93]:
root_file_path = "/groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100000.gtac.root"
processing_start_time = time.time()
prometheus_set_50k, primary_set_50k = parse_and_convert_genie(root_file_path)
prometheus_set, primary_set  = prometheus_set_50k, primary_set_50k 
processing_end_time = time.time()
print(f"File processing and conversion completed in {processing_end_time - processing_start_time:.2f} seconds")


2025-05-14 02:06:11,831 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100000.gtac.root
2025-05-14 02:06:11,833 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100000.gtac.root
2025-05-14 02:06:11,841 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100000.gtac.root
2025-05-14 02:06:11,850 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100000.gtac.root
2025-05-14 02:06:11,867 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100000.gtac.root
2025-05-14 02:06:11,867 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100000.gtac.root
2025-05-14 02:06:11,870 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/outpu

File processing and conversion completed in 48.14 seconds


In [56]:
root_file_path = "/groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_300_2.gtac.root"
processing_start_time = time.time()
prometheus_set_30k, primary_set_30k = parse_and_convert_genie(root_file_path)
processing_end_time = time.time()
print(f"File processing and conversion completed in {processing_end_time - processing_start_time:.2f} seconds")


2025-05-14 02:01:41,900 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_300_2.gtac.root
2025-05-14 02:01:41,901 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_300_2.gtac.root
2025-05-14 02:01:41,902 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_300_2.gtac.root
2025-05-14 02:01:41,902 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_300_2.gtac.root
2025-05-14 02:01:41,913 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_300_2.gtac.root
2025-05-14 02:01:41,914 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_300_2.gtac.root
2025-05-14 02:01:41,915 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_ghep

File processing and conversion completed in 0.17 seconds


/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/genie_parser_injection.py:20: RuntimeWarning: invalid value encountered in scalar divide
  angle = np.arccos(np.clip(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1))


In [94]:
prometheus_set_50k

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False, False, False, False, False, Fal...","[12.241536434950843, 1.4266409556002633, 1.001...","[13, 2112, 2212, 2112, 2112, 2112, 2112, 111]","[CC, CC, CC, CC, CC, CC, CC, CC]","[0.0351263974655324, 0.28481897729629824, 0.55...","[0.25242025638962373, 2.277800838752831, 0.829...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, chi..."
1,"[False, False, False, False, False, False, Fal...","[0.00618, 11.159845145325475, 3.62720331445329...","[22, 13, 211, 111, 111, 111, 2112, 111]","[CC, CC, CC, CC, CC, CC, CC, CC]","[2.3913877543870186, 0.132762958382109, 0.2822...","[0.5312741211053426, 0.06686440482811626, 2.82...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, chi..."
2,"[False, False, False, False, False, False]","[21.789113939956255, 1.5662808760692444, 0.389...","[13, 2112, 211, 211, 211, -211]","[CC, CC, CC, CC, CC, CC]","[0.017712434770100097, 0.09306290978763258, 0....","[0.9877718809379501, 2.4458843432421595, 1.912...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child]"
3,"[False, False, False, False, False, False, Fal...","[7.877847294092507, 1.6768188608666141, 1.3446...","[13, 211, 2112, 111, 2112, 2212, 211, -211, 111]","[CC, CC, CC, CC, CC, CC, CC, CC, CC]","[0.07368849136478363, 0.17176360568419854, 0.2...","[2.753809556364011, 0.7428464456092918, 1.0213...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
4,"[False, False, False, False, False, False, Fal...","[0.00368, 5.461169277325859, 4.706355882383697...","[22, 13, -321, 211, 311, 211, -321, 321, 2112,...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC]","[0.8988816101352612, 0.05745776780306794, 0.08...","[1.8669099522702557, 0.8426356266844667, 0.780...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,"[False, False, False, False, False, False, Fal...","[22.145512223674842, 7.743279937131325, 1.9751...","[13, 211, 311, 111, 3122, 111, 211, 111]","[CC, CC, CC, CC, CC, CC, CC, CC]","[0.09115578644226212, 0.12363475534324363, 0.2...","[0.274825536116918, 3.047922659153318, 2.65788...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [95]:
prometheus_set_50k

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False, False, False, False, False, Fal...","[12.241536434950843, 1.4266409556002633, 1.001...","[13, 2112, 2212, 2112, 2112, 2112, 2112, 111]","[CC, CC, CC, CC, CC, CC, CC, CC]","[0.0351263974655324, 0.28481897729629824, 0.55...","[0.25242025638962373, 2.277800838752831, 0.829...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, chi..."
1,"[False, False, False, False, False, False, Fal...","[0.00618, 11.159845145325475, 3.62720331445329...","[22, 13, 211, 111, 111, 111, 2112, 111]","[CC, CC, CC, CC, CC, CC, CC, CC]","[2.3913877543870186, 0.132762958382109, 0.2822...","[0.5312741211053426, 0.06686440482811626, 2.82...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, chi..."
2,"[False, False, False, False, False, False]","[21.789113939956255, 1.5662808760692444, 0.389...","[13, 2112, 211, 211, 211, -211]","[CC, CC, CC, CC, CC, CC]","[0.017712434770100097, 0.09306290978763258, 0....","[0.9877718809379501, 2.4458843432421595, 1.912...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child]"
3,"[False, False, False, False, False, False, Fal...","[7.877847294092507, 1.6768188608666141, 1.3446...","[13, 211, 2112, 111, 2112, 2212, 211, -211, 111]","[CC, CC, CC, CC, CC, CC, CC, CC, CC]","[0.07368849136478363, 0.17176360568419854, 0.2...","[2.753809556364011, 0.7428464456092918, 1.0213...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
4,"[False, False, False, False, False, False, Fal...","[0.00368, 5.461169277325859, 4.706355882383697...","[22, 13, -321, 211, 311, 211, -321, 321, 2112,...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC]","[0.8988816101352612, 0.05745776780306794, 0.08...","[1.8669099522702557, 0.8426356266844667, 0.780...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,"[False, False, False, False, False, False, Fal...","[22.145512223674842, 7.743279937131325, 1.9751...","[13, 211, 311, 111, 3122, 111, 211, 111]","[CC, CC, CC, CC, CC, CC, CC, CC]","[0.09115578644226212, 0.12363475534324363, 0.2...","[0.274825536116918, 3.047922659153318, 2.65788...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [96]:
prometheus_set_30k

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False, False, False, False, False, False]","[15.801941044461033, 1.5981514164364086, 0.967...","[13, 2212, 2212, 2212, 2212, 2112, 2112]","[CC, CC, CC, CC, CC, CC, CC]","[0.025403922430313617, 0.1136431400562615, 2.7...","[2.3396360396245264, 1.3002280674356184, 1.348...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
1,"[False, False, False, False, False, False, False]","[0.00632, 21.888680580700242, 2.59497243382819...","[22, 14, 111, -211, 111, 2212, 211]","[NC, NC, NC, NC, NC, NC, NC]","[3.053783025736545, 0.1039251807881697, 0.0059...","[1.6769714728901948, 0.47860600954939597, 1.74...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
2,"[False, False, False, False, False, False, Fal...","[0.00618, 15.468053714931214, 0.66084979313053...","[22, 13, 111, 211, 311, 2112, -321, 211, 111, ...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC]","[1.2684001785382295, 0.03602436517357272, 0.18...","[2.5440440410515768, 0.5674743827649306, 1.822...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
3,"[False, False, False]","[0.00618, 47.903668427315985, 2.4322445468525333]","[22, 13, 2212]","[CC, CC, CC]","[2.0237879310050473, 0.029086804979322794, 0.7...","[3.0272232995671318, 3.073927743666174, 0.1622...","[-1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0]","[child, child, child]"
4,"[False, False, False, False, False]","[12.928301273896695, 2.1789688929221374, 1.367...","[13, 2212, 111, 111, 111]","[CC, CC, CC, CC, CC]","[0.04486957689963508, 0.0114732421629151, 0.06...","[1.8619716952128154, 1.5469823378515137, 1.025...","[-1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"[False, False, False, False, False, False, Fal...","[8.644147801983845, 1.6319309694163056, 5.9133...","[13, -321, 311, 211, 2212, 2212, 2212, 2212, 2...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, C...","[0.1613836671210928, 0.08306223639053313, 0.13...","[2.2448064604543925, 1.1983356410760624, 0.799...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
296,"[False, False, False, False, False, False]","[19.182006607170106, 3.397708107918532, 0.8687...","[13, 3222, 321, -211, 2212, 111]","[CC, CC, CC, CC, CC, CC]","[0.042927

In [60]:
prometheus_set_30k

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False, False, False, False, False, False]","[15.801941044461033, 1.5981514164364086, 0.967...","[13, 2212, 2212, 2212, 2212, 2112, 2112]","[CC, CC, CC, CC, CC, CC, CC]","[0.025403922430313617, 0.1136431400562615, 2.7...","[2.3396360396245264, 1.3002280674356184, 1.348...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
1,"[False, False, False, False, False, False, False]","[0.00632, 21.888680580700242, 2.59497243382819...","[22, 14, 111, -211, 111, 2212, 211]","[NC, NC, NC, NC, NC, NC, NC]","[3.053783025736545, 0.1039251807881697, 0.0059...","[1.6769714728901948, 0.47860600954939597, 1.74...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
2,"[False, False, False, False, False, False, Fal...","[0.00618, 15.468053714931214, 0.66084979313053...","[22, 13, 111, 211, 311, 2112, -321, 211, 111, ...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC]","[1.2684001785382295, 0.03602436517357272, 0.18...","[2.5440440410515768, 0.5674743827649306, 1.822...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
3,"[False, False, False]","[0.00618, 47.903668427315985, 2.4322445468525333]","[22, 13, 2212]","[CC, CC, CC]","[2.0237879310050473, 0.029086804979322794, 0.7...","[3.0272232995671318, 3.073927743666174, 0.1622...","[-1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0]","[child, child, child]"
4,"[False, False, False, False, False]","[12.928301273896695, 2.1789688929221374, 1.367...","[13, 2212, 111, 111, 111]","[CC, CC, CC, CC, CC]","[0.04486957689963508, 0.0114732421629151, 0.06...","[1.8619716952128154, 1.5469823378515137, 1.025...","[-1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"[False, False, False, False, False, False, Fal...","[8.644147801983845, 1.6319309694163056, 5.9133...","[13, -321, 311, 211, 2212, 2212, 2212, 2212, 2...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, C...","[0.1613836671210928, 0.08306223639053313, 0.13...","[2.2448064604543925, 1.1983356410760624, 0.799...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
296,"[False, False, False, False, False, False]","[19.182006607170106, 3.397708107918532, 0.8687...","[13, 3222, 321, -211, 2212, 111]","[CC, CC, CC, CC, CC, CC]","[0.042927

In [61]:
prometheus_set_30k.iloc[0], prometheus_set_50k.iloc[0]

(primary         [False, False, False, False, False, False, False]
 e               [15.801941044461033, 1.5981514164364086, 0.967...
 pdg_code                 [13, 2212, 2212, 2212, 2212, 2112, 2112]
 interaction                          [CC, CC, CC, CC, CC, CC, CC]
 theta           [0.025403922430313617, 0.1136431400562615, 2.7...
 phi             [2.3396360396245264, 1.3002280674356184, 1.348...
 bjorken_x              [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
 bjorken_y              [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
 pos_x                         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 pos_y                         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 pos_z                         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 position        [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...
 column_depth           [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
 custom_info     [child, child, child, child, child, child, child]
 Name: 0, dtype: object,
 primary                [False, False

In [62]:


np.array(prometheus_set['pdg_code'])[0][1]

13

In [97]:
len(prometheus_set)

100000

In [64]:
# sss = []
# for i in range(len(np.array(prometheus_set['pdg_code']))):
#     if i % 500 == 0:
#         print(i)
#     for j in range(len(np.array(prometheus_set['pdg_code'])[i])):

#         sss.append(np.array(prometheus_set['pdg_code'])[i][j])


In [98]:
# This is much faster than nested loops
pdg_codes = []
for event_pdg_codes in prometheus_set['pdg_code']:
    pdg_codes.extend(event_pdg_codes)

# Or even better, using list comprehension
pdg_codes = [code for event_pdg_codes in prometheus_set['pdg_code'] for code in event_pdg_codes]

# If you need to see progress, you can still do that
pdg_codes = []
total_events = len(prometheus_set['pdg_code'])
for i, event_pdg_codes in enumerate(prometheus_set['pdg_code']):
    if i % 500 == 0:
        print(f"Processing event {i}/{total_events}")
    pdg_codes.extend(event_pdg_codes)

Processing event 0/100000
Processing event 500/100000
Processing event 1000/100000
Processing event 1500/100000
Processing event 2000/100000
Processing event 2500/100000
Processing event 3000/100000
Processing event 3500/100000
Processing event 4000/100000
Processing event 4500/100000
Processing event 5000/100000
Processing event 5500/100000
Processing event 6000/100000
Processing event 6500/100000
Processing event 7000/100000
Processing event 7500/100000
Processing event 8000/100000
Processing event 8500/100000
Processing event 9000/100000
Processing event 9500/100000
Processing event 10000/100000
Processing event 10500/100000
Processing event 11000/100000
Processing event 11500/100000
Processing event 12000/100000
Processing event 12500/100000
Processing event 13000/100000
Processing event 13500/100000
Processing event 14000/100000
Processing event 14500/100000
Processing event 15000/100000
Processing event 15500/100000
Processing event 16000/100000
Processing event 16500/100000
Proc

In [86]:
prometheus_set['pdg_code'][3646]

array([  13,  211, -211, 2212,  211], dtype=int32)

In [87]:
prometheus_set['e'][3646]

array([11.47653205,  3.56056348,  0.17394038,  1.1832433 ,  0.3093494 ])

In [75]:
simset = 192
start_time = time.time()

print(f"Using simset: {simset}")
print(f"Using root file: {root_file_path}")

# Create subsets from the working copies
processing_start_time = time.time()
prometheus_subset = prometheus_set_copy.iloc[3645:3649].copy()
primary_subset = primary_set_copy.iloc[3645:3649].copy()
processing_end_time = time.time()
print(f"Subset creation completed in {processing_end_time - processing_start_time:.2f} seconds")

# Define output paths
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
primrary_file_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet'
prometheus_file_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'

# Timing for serialization and saving
save_start_time = time.time()

# Convert position arrays to serializable format
prometheus_subset['position'] = prometheus_subset['position'].apply(
    lambda x: [[float(val) for val in arr] for arr in x]
)
primary_subset['position'] = primary_subset['position'].apply(
    lambda x: [float(val) for val in x]
)

# Save to parquet
num_events = len(prometheus_subset)
print(f"Processing {num_events} events")
prometheus_subset.to_parquet(prometheus_file_path)
primary_subset.to_parquet(primrary_file_path)
save_end_time = time.time()
print(f"Serialization and saving completed in {save_end_time - save_start_time:.2f} seconds")

# GENIE stuff
config["injection"]["name"] = "GENIE"
config["run"]["outfile"] = f"{OUTPUT_DIR}/100_events_{timestamp}.parquet"
config["run"]["nevents"] = num_events
config["injection"]["GENIE"] = config["injection"].get("GENIE", {})
config["injection"]["GENIE"]["paths"] = config["injection"]["GENIE"].get("paths", {})
config["injection"]["GENIE"]["inject"] = False  # we aren't injecting using prometheus
config["injection"]["GENIE"]["simulation"] = {}

# geofile
config["detector"]["geo file"] = f"{RESOURCE_DIR}/geofiles/icecube.geo"

# ppc configuration
config['photon propagator']['name'] = 'PPC_UPGRADE'
config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpdir"] = "./ppc_tmpdir"+str(simset)
config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpfile"] = "ppc_tmp"+str(simset)
config["photon propagator"]["PPC_UPGRADE"]["simulation"]["supress_output"] = True  # for printing!

# Timing for Prometheus simulation
sim_start_time = time.time()
p = Prometheus(config, primary_set_parquet_path=primrary_file_path, prometheus_set_parquet_path=prometheus_file_path)
p.sim()
sim_end_time = time.time()
print(f"Prometheus simulation completed in {sim_end_time - sim_start_time:.2f} seconds")

# End timing
end_time = time.time()
total_time = end_time - start_time
print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
print('Finished without catastrophic error')

Using simset: 192
Using root file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_300_2.gtac.root


NameError: name 'prometheus_set_copy' is not defined

In [76]:
prometheus_subset

NameError: name 'prometheus_subset' is not defined

In [77]:
primary_set = primary_set_copy.copy()
prometheus_set = prometheus_set_copy.copy()


NameError: name 'primary_set_copy' is not defined

In [78]:
# 1. Create the subsets
prometheus_subset = prometheus_set_copy.iloc[3644:3649].copy()
primary_subset = primary_set_copy.iloc[3644:3649].copy()

# 2. Convert position column to serializable format
prometheus_subset['position'] = prometheus_subset['position'].apply(
    lambda x: [[float(val) for val in arr] for arr in x]
)

primary_subset['position'] = primary_subset['position'].apply(
    lambda x: [float(val) for val in x]
)

# 3. REMOVE the position_original column if it exists
if 'position_original' in prometheus_subset.columns:
    prometheus_subset = prometheus_subset.drop('position_original', axis=1)
    
if 'position_original' in primary_subset.columns:
    primary_subset = primary_subset.drop('position_original', axis=1)

# 4. Save to parquet
primrary_file_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet'
prometheus_file_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'
primary_subset.to_parquet(primrary_file_path)
prometheus_subset.to_parquet(prometheus_file_path)

# 5. Continue with your simulation

NameError: name 'prometheus_set_copy' is not defined

In [79]:
prometheus_subset = prometheus_set.iloc[3644:3649].copy()  # Create a copy to avoid modifying the original
primary_subset = primary_set.iloc[3644:3649].copy()

In [229]:
primary_subset

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info


In [243]:
prometheus_subset

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
3644,"[False, False, False, False, False, False, Fal...","[0.8542792550526659, 24.08363411684848, 0.8725...","[13, 421, 211, -211, 111, 211, 111, -211, 111,...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, C...","[0.03503344295312818, 0.024472056642438362, 0....","[1.6569847610790494, 3.007676113342956, 2.5751...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
3645,"[False, False, False, False, False, False, Fal...","[13.405844228702241, 4.687262258622975, 0.2562...","[14, 111, -211, 2212, 211, 111, 2112, -211]","[NC, NC, NC, NC, NC, NC, NC, NC]","[0.09181583703946351, 0.23972122101472812, 0.5...","[2.617173842071726, 0.32048861088750497, 0.703...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, chi..."
3646,"[False, False, False, False, False, False, Fal...","[1.0795339975313, 7.074191149181425, 0.1203815...","[13, 22, 22, 11, -11, 3122, 111, 311, 211, -32...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC]","[0.16628440472561506, 0.01946900342643551, 0.0...","[2.043475696809527, 1.2860429203417139, 1.5597...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
3647,"[False, False, False, False, False, False, Fal...","[2.4552800359511826, 0.8625432694193421, 2.849...","[14, 311, -311, 111, 211, -211, 211, 111, 22, ...","[NC, NC, NC, NC, NC, NC, NC, NC, NC, NC, NC, N...","[0.012720933634840297, 0.13819189865805373, 0....","[1.6149440389128824, 2.3683809102154365, 0.276...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
3648,"[False, False, False, False, False, False, Fal...","[0.00618, 2.790348965048556, 5.585656311888313...","[22, 13, 111, 2112, 211, -211, 2212, 211, 2112...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC]","[2.935142770984232, 0.40117513197430293, 0.080...","[1.0294412386682419, 0.7909117241497265, 2.220...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."


In [242]:
primary_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
3644,True,48.936735,14,CC,0.0,NaN,-1,-1,0.0,0.0,0.0,"[0.0, 0.0, 0.0]",-1,nu:14;tgt:1000080160;N:2212;q:1(v);proc:Weak[C...
3645,True,22.914816,14,NC,0.0,NaN,-1,-1,0.0,0.0,0.0,"[0.0, 0.0, 0.0]",-1,nu:14;tgt:1000080160;N:2112;q:-2(s);proc:Weak[...
3646,True,34.763364,14,CC,0.0,NaN,-1,-1,0.0,0.0,0.0,"[0.0, 0.0, 0.0]",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
3647,True,25.558242,14,NC,0.0,NaN,-1,-1,0.0,0.0,0.0,"[0.0, 0.0, 0.0]",-1,nu:14;tgt:1000010010;N:2212;q:1(s);proc:Weak[N...
3648,True,13.868264,14,CC,0.0,NaN,-1,-1,0.0,0.0,0.0,"[0.0, 0.0, 0.0]",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...


In [240]:
# # Convert the position column from arrays to simple numpy arrays before saving
# def convert_position(pos_array):
#     if isinstance(pos_array, np.ndarray) and len(pos_array) > 0:
#         # If it's already a numpy array, just return the first element as a regular array
#         return pos_array[0].tolist() if pos_array.shape[0] > 0 else [0, 0, 0]
#     elif isinstance(pos_array, list) and len(pos_array) > 0:
#         # If it's a list of arrays, return the first element as a list
#         return pos_array[0] if isinstance(pos_array[0], list) else [0, 0, 0]
#     else:
#         # Default safe value
#         return [0, 0, 0]

# # Apply conversion to both DataFrames
# prometheus_subset['position'] = prometheus_subset['position'].apply(convert_position)
# primary_subset['position'] = primary_subset['position'].apply(convert_position)

# # Now update and save
prometheus_set, primary_set = prometheus_subset.copy(), primary_subset.copy()


In [241]:
simset = 192
start_time = time.time()


print(f"Using simset: {simset}")
print(f"Using root file: {root_file_path}")

# Timing for file processing and conversion in one step
processing_start_time = time.time()
# prometheus_set, primary_set = primary_subset,prometheus_subset
processing_end_time = time.time()
print(f"File processing and conversion completed in {processing_end_time - processing_start_time:.2f} seconds")

# absolute path:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# output_dir = f"/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/examples/output"
primrary_file_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet'
prometheus_file_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'

# # inject into cylinder:
# primary_set, prometheus_set = inject_particles_in_cylinder(primary_set, ## neutrino information
#                     prometheus_set, ## child particle information
#                     cylinder_radius=500, ## meters
#                     cylinder_height = 1000, # meters
#                     cylinder_center = (0, 0, 0), # meters not including offset
#                     )

#primary_set, prometheus_set = rotate_particles_final(primary_set, prometheus_set)


# Timing for serialization and saving
save_start_time = time.time()
# position arrays to serialized strings
# prometheus_subset['position'] = prometheus_subset['position'].apply(lambda x: [arr.tolist() for arr in x])

# # Now assign these to your main variables
# prometheus_set, primary_set = primary_subset, prometheus_subset
prometheus_set['position'] = prometheus_set['position'].apply(lambda x: [arr.tolist() for arr in x])
num_events = len(prometheus_set)
print(f"Processing {num_events} events")
prometheus_set.to_parquet(prometheus_file_path)
primary_set.to_parquet(primrary_file_path)
save_end_time = time.time()
print(f"Serialization and saving completed in {save_end_time - save_start_time:.2f} seconds")

## GENIE stuff:
config["injection"]["name"] = "GENIE"
config["run"]["outfile"] = f"{OUTPUT_DIR}/100_events_{timestamp}.parquet"
config["run"]["nevents"] = num_events
config["injection"]["GENIE"] = config["injection"].get("GENIE", {})
config["injection"]["GENIE"]["paths"] = config["injection"]["GENIE"].get("paths", {})
config["injection"]["GENIE"]["inject"] = False ## we aren't injecting using prometheus, we are using an injection file
config["injection"]["GENIE"]["simulation"] = {}
## geofile:
config["detector"]["geo file"] = f"{RESOURCE_DIR}/geofiles/icecube.geo"
## ppc configuration:
config['photon propagator']['name'] = 'PPC_UPGRADE'
config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpdir"] = "./ppc_tmpdir"+str(simset)
config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpfile"] = "ppc_tmp"+str(simset)
config["photon propagator"]["PPC_UPGRADE"]["simulation"]["supress_output"] = False ## for printing!

# Timing for Prometheus simulation
sim_start_time = time.time()
p = Prometheus(config, primary_set_parquet_path=primrary_file_path, prometheus_set_parquet_path=prometheus_file_path)
p.sim()
sim_end_time = time.time()
print(f"Prometheus simulation completed in {sim_end_time - sim_start_time:.2f} seconds")

# End timing
end_time = time.time()
total_time = end_time - start_time
print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
print('Finished without catastrophic error')




Using simset: 192
Using root file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_50k.gtac.root
File processing and conversion completed in 0.00 seconds


AttributeError: 'list' object has no attribute 'tolist'

In [185]:
prometheus_subset

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
3644,"[False, False, False, False, False, False, Fal...","[0.8542792550526659, 24.08363411684848, 0.8725...","[13, 421, 211, -211, 111, 211, 111, -211, 111,...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, C...","[0.03503344295312818, 0.024472056642438362, 0....","[1.6569847610790494, 3.007676113342956, 2.5751...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
3645,"[False, False, False, False, False, False, Fal...","[13.405844228702241, 4.687262258622975, 0.2562...","[14, 111, -211, 2212, 211, 111, 2112, -211]","[NC, NC, NC, NC, NC, NC, NC, NC]","[0.09181583703946351, 0.23972122101472812, 0.5...","[2.617173842071726, 0.32048861088750497, 0.703...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, chi..."
3646,"[False, False, False, False, False, False, Fal...","[1.0795339975313, 7.074191149181425, 0.1203815...","[13, 22, 22, 11, -11, 3122, 111, 311, 211, -32...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC]","[0.16628440472561506, 0.01946900342643551, 0.0...","[2.043475696809527, 1.2860429203417139, 1.5597...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
3647,"[False, False, False, False, False, False, Fal...","[2.4552800359511826, 0.8625432694193421, 2.849...","[14, 311, -311, 111, 211, -211, 211, 111, 22, ...","[NC, NC, NC, NC, NC, NC, NC, NC, NC, NC, NC, N...","[0.012720933634840297, 0.13819189865805373, 0....","[1.6149440389128824, 2.3683809102154365, 0.276...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
3648,"[False, False, False, False, False, False, Fal...","[0.00618, 2.790348965048556, 5.585656311888313...","[22, 13, 111, 2112, 211, -211, 2212, 211, 2112...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC]","[2.935142770984232, 0.40117513197430293, 0.080...","[1.0294412386682419, 0.7909117241497265, 2.220...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."


In [88]:
pdg_codes

[14,
 2112,
 111,
 2212,
 13,
 111,
 211,
 2212,
 111,
 13,
 2212,
 2212,
 2212,
 2212,
 2112,
 2112,
 2112,
 2212,
 13,
 -211,
 211,
 111,
 321,
 -211,
 -321,
 111,
 2212,
 211,
 14,
 111,
 211,
 2112,
 111,
 13,
 2112,
 211,
 111,
 13,
 211,
 2112,
 211,
 22,
 13,
 111,
 2112,
 2212,
 2112,
 211,
 22,
 14,
 3122,
 311,
 211,
 -211,
 111,
 111,
 22,
 13,
 211,
 2212,
 2212,
 2212,
 -211,
 2112,
 211,
 111,
 2112,
 2112,
 211,
 111,
 2112,
 211,
 22,
 13,
 111,
 111,
 211,
 2212,
 -211,
 13,
 2112,
 211,
 13,
 111,
 -211,
 211,
 111,
 211,
 -211,
 2212,
 211,
 211,
 -211,
 14,
 111,
 211,
 111,
 2112,
 -211,
 2212,
 211,
 -211,
 2212,
 2212,
 2212,
 2212,
 2212,
 2112,
 2212,
 -211,
 111,
 13,
 -211,
 211,
 111,
 2112,
 2112,
 311,
 111,
 2212,
 2212,
 111,
 2112,
 13,
 211,
 111,
 311,
 -321,
 2212,
 111,
 -211,
 211,
 13,
 2112,
 2212,
 22,
 13,
 -211,
 211,
 111,
 2212,
 211,
 211,
 -211,
 22,
 13,
 111,
 2212,
 -211,
 111,
 2212,
 211,
 211,
 2212,
 -211,
 13,
 111,
 111,
 2212,
 2

In [99]:
prometheus_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False, False, False, False, False, Fal...","[12.241536434950843, 1.4266409556002633, 1.001...","[13, 2112, 2212, 2112, 2112, 2112, 2112, 111]","[CC, CC, CC, CC, CC, CC, CC, CC]","[0.0351263974655324, 0.28481897729629824, 0.55...","[0.25242025638962373, 2.277800838752831, 0.829...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, chi..."
1,"[False, False, False, False, False, False, Fal...","[0.00618, 11.159845145325475, 3.62720331445329...","[22, 13, 211, 111, 111, 111, 2112, 111]","[CC, CC, CC, CC, CC, CC, CC, CC]","[2.3913877543870186, 0.132762958382109, 0.2822...","[0.5312741211053426, 0.06686440482811626, 2.82...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, chi..."
2,"[False, False, False, False, False, False]","[21.789113939956255, 1.5662808760692444, 0.389...","[13, 2112, 211, 211, 211, -211]","[CC, CC, CC, CC, CC, CC]","[0.017712434770100097, 0.09306290978763258, 0....","[0.9877718809379501, 2.4458843432421595, 1.912...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child]"
3,"[False, False, False, False, False, False, Fal...","[7.877847294092507, 1.6768188608666141, 1.3446...","[13, 211, 2112, 111, 2112, 2212, 211, -211, 111]","[CC, CC, CC, CC, CC, CC, CC, CC, CC]","[0.07368849136478363, 0.17176360568419854, 0.2...","[2.753809556364011, 0.7428464456092918, 1.0213...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
4,"[False, False, False, False, False, False, Fal...","[0.00368, 5.461169277325859, 4.706355882383697...","[22, 13, -321, 211, 311, 211, -321, 321, 2112,...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC]","[0.8988816101352612, 0.05745776780306794, 0.08...","[1.8669099522702557, 0.8426356266844667, 0.780...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,"[False, False, False, False, False, False, Fal...","[22.145512223674842, 7.743279937131325, 1.9751...","[13, 211, 311, 111, 3122, 111, 211, 111]","[CC, CC, CC, CC, CC, CC, CC, CC]","[0.09115578644226212, 0.12363475534324363, 0.2...","[0.274825536116918, 3.047922659153318, 2.65788...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [100]:
np.unique(pdg_codes)

array([     -3122,      -2212,      -2112,       -321,       -311,
             -211,        -13,        -11,         11,         12,
               13,         14,         22,        111,        130,
              211,        311,        321,        411,        421,
              431,       2112,       2212,       3112,       3122,
             3212,       3222,       4122,       4212,       4222,
       1000080160, 2000000101], dtype=int32)

In [90]:
np.unique(pdg_codes)

array([     -3122,      -2212,      -2112,       -321,       -311,
             -211,        -13,        -11,         11,         12,
               13,         14,         22,        111,        130,
              211,        311,        321,        411,        421,
              431,       2112,       2212,       3112,       3122,
             3212,       3222,       4122,       4212,       4222,
       1000080160, 2000000101], dtype=int32)

In [39]:
# Method 1: Create a list of tuples (event_number, pdg_code)
event_pdg_pairs = []
for event_num, event_pdg_codes in enumerate(prometheus_set['pdg_code']):
    for code in event_pdg_codes:
        event_pdg_pairs.append((event_num, code))

# Method 2: Using list comprehension for the same result
event_pdg_pairs = [(event_num, code) for event_num, event_pdg_codes in enumerate(prometheus_set['pdg_code']) 
                   for code in event_pdg_codes]

# Method 3: Create a dictionary mapping PDG codes to lists of event numbers
pdg_to_events = {}
for event_num, event_pdg_codes in enumerate(prometheus_set['pdg_code']):
    for code in event_pdg_codes:
        if code not in pdg_to_events:
            pdg_to_events[code] = []
        pdg_to_events[code].append(event_num)

# Method 4: With progress tracking
event_pdg_pairs = []
total_events = len(prometheus_set['pdg_code'])
for event_num, event_pdg_codes in enumerate(prometheus_set['pdg_code']):
    if event_num % 500 == 0:
        print(f"Processing event {event_num}/{total_events}")
    for code in event_pdg_codes:
        event_pdg_pairs.append((event_num, code))

Processing event 0/10000
Processing event 500/10000
Processing event 1000/10000
Processing event 1500/10000
Processing event 2000/10000
Processing event 2500/10000
Processing event 3000/10000
Processing event 3500/10000
Processing event 4000/10000
Processing event 4500/10000
Processing event 5000/10000
Processing event 5500/10000
Processing event 6000/10000
Processing event 6500/10000
Processing event 7000/10000
Processing event 7500/10000
Processing event 8000/10000
Processing event 8500/10000
Processing event 9000/10000
Processing event 9500/10000


In [40]:
# If using Method 1 or 2 (event_pdg_pairs list)
events_with_specific_pdg = [event_num for event_num, code in event_pdg_pairs if code == 2000000101]

# If using Method 3 (pdg_to_events dictionary) - this is much faster for lookups
events_with_specific_pdg = pdg_to_events.get(2000000101, [])

print(f"PDG code 2000000101 appears in events: {events_with_specific_pdg}")

PDG code 2000000101 appears in events: [153, 616, 616, 1012, 1342, 1342, 1361, 1361, 1372, 1423, 1876, 1971, 2132, 2227, 2227, 2316, 2342, 2400, 2406, 2469, 2469, 2628, 2769, 2793, 2812, 2928, 3006, 3169, 3208, 3325, 3351, 3617, 3671, 3671, 3811, 3949, 4343, 4693, 4693, 4876, 4876, 5757, 5784, 5873, 6066, 6266, 6266, 6346, 6424, 6730, 6730, 6742, 6814, 6835, 6835, 6929, 7444, 7444, 7457, 7539, 7872, 8020, 8046, 8046, 8220, 8907, 8970, 8970, 9433, 9433, 9444, 9685, 9685]


In [41]:
prometheus_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False, False, False, False, False, Fal...","[45.50518012924548, 4.083093586966454, 1.09807...","[13, 111, 2112, 2112, 211, 111, 2112, 2212, 21...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC]","[0.021449077285086746, 0.05519605740903047, 1....","[0.40517244251704004, 2.2834485776412294, 0.32...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
1,"[False, False, False, False, False, False, Fal...","[0.00632, 1.553154902475276, 6.857837626037752...","[22, 13, 211, 2212, 2112, 2112, 2112, 2112, 21...","[CC, CC, CC, CC, CC, CC, CC, CC, CC, CC, CC]","[1.1951293419965532, 0.4609043611329849, 0.060...","[0.27363585115154915, 0.42048033485688263, 2.4...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
2,"[False, False, False, False, False, False]","[22.8860991194759, 3.6412743086031933, 0.71695...","[14, 2112, 211, 111, 211, -211]","[NC, NC, NC, NC, NC, NC]","[0.10184900779281678, 0.2221679576074078, 0.69...","[2.8634529971675935, 0.0007990003171211767, 0....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child]"
3,"[False, False, False, False, False, False, Fal...","[0.00618, 0.6646495645253374, 17.8186436138155...","[22, 14, 211, -211, 211, 111, 2112, 2112, 2112...","[NC, NC, NC, NC, NC, NC, NC, NC, NC, NC, NC, NC]","[1.5412930543401684, 0.6991168348066554, 0.008...","[0.8213070164024534, 2.1867666481450447, 0.822...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
4,"[False, False, False, False, False]","[22.03573692056882, 2.3611988089802787, 0.6842...","[14, 2212, 211, 111, 2112]","[NC, NC, NC, NC, NC]","[0.056240052732994414, 0.4125022777293858, 0.8...","[2.489758917484508, 0.9283320982002338, 0.2898...","[-1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,"[False, False, False, False, False]","[0.00618, 27.600063236991506, 0.98794713063116...","[22, 13, 2212, 211, -211]","[CC, CC, CC, CC, CC]","[2.6153140021378816, 0.008360050616784829, 0.5...","[1.3115869237856763, 1.1528934813206155, 1.750...","[-1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child]"
9996,"[False, False, False, Fa

In [42]:
prometheus_set.columns

Index(['primary', 'e', 'pdg_code', 'interaction', 'theta', 'phi', 'bjorken_x',
       'bjorken_y', 'pos_x', 'pos_y', 'pos_z', 'position', 'column_depth',
       'custom_info'],
      dtype='object')

In [45]:
# Examine event 153 (the first one where our code appears)
event_num = 153
print(f"Examining Event {event_num}:")

# Get all PDG codes for this event
event_pdg_codes = prometheus_set['pdg_code'][event_num]
print(f"PDG codes in event: {event_pdg_codes}")

# Print other relevant information
print(f"Primary: {prometheus_set['primary'][event_num]}")
print(f"Energy: {prometheus_set['e'][event_num]}")
print(f"Interaction type: {prometheus_set['interaction'][event_num]}")

# Try to find the index of our code in the array
indices = np.where(event_pdg_codes == 2000000101)[0]
print(f"Indices of 2000000101 in the array: {indices}")

Examining Event 153:
PDG codes in event: [        14       2212 2000000101]
Primary: [False False False]
Energy: [10.5191234   0.98778255  0.027     ]
Interaction type: ['NC' 'NC' 'NC']
Indices of 2000000101 in the array: [2]


In [49]:
prometheus_set.iloc[616]

primary                       [False, False, False, False, False]
e               [21.155983874077165, 0.9269753750557247, 1.363...
pdg_code                 [14, 2112, 2212, 2000000101, 2000000101]
interaction                                  [NC, NC, NC, NC, NC]
theta           [0.012213825755005086, nan, 0.2234443573612825...
phi             [1.8530011722666728, nan, 1.6908721163309737, ...
bjorken_x                          [-1.0, -1.0, -1.0, -1.0, -1.0]
bjorken_y                          [-1.0, -1.0, -1.0, -1.0, -1.0]
pos_x                                   [0.0, 0.0, 0.0, 0.0, 0.0]
pos_y                                   [0.0, 0.0, 0.0, 0.0, 0.0]
pos_z                                   [0.0, 0.0, 0.0, 0.0, 0.0]
position        [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...
column_depth                       [-1.0, -1.0, -1.0, -1.0, -1.0]
custom_info                   [child, child, child, child, child]
Name: 616, dtype: object

In [50]:
primary_set.iloc[616]

primary                                                   True
e                                                    21.645134
pdg_code                                                    14
interaction                                                 NC
theta                                                      0.0
phi                                                        NaN
bjorken_x                                                   -1
bjorken_y                                                   -1
pos_x                                                      0.0
pos_y                                                      0.0
pos_z                                                      0.0
position                                       [0.0, 0.0, 0.0]
column_depth                                                -1
custom_info     nu:14;tgt:1000080160;N:2112;proc:Weak[NC],QES;
Name: 616, dtype: object